<a href="https://colab.research.google.com/github/neylsoncrepalde/optimization_exercises/blob/main/modelagem_prescritiva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelagem Prescritiva

Curso ministrado pelo prof. **Claudio Lucio** na **A3Data** em 08-12-2020.

## Retomada histórica

- Desenvolvimento de 2 problemas insolúveis na matemática por George Dantzig, então aluno de doutorado. 
- Dantzig desenvolveu o algoritmo *simplex*.

## Quiz - Rota de entrega

In [1]:
60 + 80 + 35 + 70

245

In [2]:
105 + 35 + 115 + 60

315

In [3]:
70 + 115 + 80 + 105

370

## Problema da Fábrica de Automóveis

**Seucarro Inc.** deve produzir 1000 automóveis Beta. A empresa tem 4 fábricas. Devido a diferenças na mão de obra e avanços tecnológicos, as plantas diferem no **custo de produção unitário** de cada carro.

Elas também utilizam diferentes quantidades de *matéria-prima* e *mão de obra*. O custo de operação, o *tempo* necessário de mão de obra e o custo de matéria prima para produzir uma unidade de cada carro em cada uma das fábricas estão evidenciados na tabela a seguir.

| Fábrica | Custo Unitário em R$1.000,00 | Mão de Obra (horas de fabricação) | Matéria-prima (unidades de material) |
|---------|------------------------------|-----------------------------------|--------------------------------------|
| 1       | 15                           | 2                                 | 3                                    |
| 2       | 10                           | 3                                 | 4                                    |
| 3       | 9                            | 4                                 | 5                                    |
| 4       | 7                            | 5                                 | 6                                    |


- Existem 3200 horas de mão de obra no total;
- Existem 4000 unidades de material que podem ser alocados às quatro fábricas;
- Um acordo trabalhista assinado requer que pelo menos 250 carros sejam produzidos na fábrica 3.

**Decisão**: Como produzir os 1000 carros com o menor custo?



In [5]:
!pip install ortools
from ortools.linear_solver import pywraplp

     |████████████████████████████████| 13.7MB 291kB/s 
     |████████████████████████████████| 1.0MB 41.4MB/s 
  Found existing installation: protobuf 3.12.4
    Uninstalling protobuf-3.12.4:
      Successfully uninstalled protobuf-3.12.4


In [6]:
#Número de fábricas
num_fabricas = 4

#Total de carros a serem produzidos
total_carros = 1000

#Custo de produção por carro em cada fábrica
Custo_unitario = [15 ,10, 9, 7 ]

#Horas de mão de obra necessárias por carro em cada fábrica
Horas_mao_obra = [2 ,3, 4, 5 ]

#Materia prima necessária por carro em cada fábrica
Materia_prima = [3 ,4, 5, 6 ]


In [12]:
# Criando o nosso modelo
modelo_fabrica = pywraplp.Solver('Fabrica de automoveis',pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

#Criando as variáveis de decisão
fabrica = {}
for i in range(num_fabricas):
    fabrica[i] = modelo_fabrica.IntVar(0, total_carros, 'fabrica[%i]' % (i))
print(fabrica)

#Criando a nossa função objetivo
custo_producao =  modelo_fabrica.Sum([Custo_unitario[i] * fabrica[i]   for i in range(num_fabricas)])    

#Atribuindo a minimização
objetivo = modelo_fabrica.Minimize(custo_producao)


### Restrições para o processo produtivo


#Na Fabrica 3 temos que produzir pelo menos 250 carros 
modelo_fabrica.Add(fabrica[2] >=250)

#Temos no total disponível 3200 horas de mão de obra como componente de custo 
modelo_fabrica.Add(modelo_fabrica.Sum([Horas_mao_obra[i] * fabrica[i]   for i in range(num_fabricas)])  <= 3200)

#Temos no total disponível 4000 unidade de material 
modelo_fabrica.Add(modelo_fabrica.Sum([Materia_prima[i] * fabrica[i]   for i in range(num_fabricas)])  <= 4000)

#Número máximo de carros a ser construído = 1000
modelo_fabrica.Add(modelo_fabrica.Sum([ fabrica[i]   for i in range(num_fabricas)])  == 1000)


{0: fabrica[0], 1: fabrica[1], 2: fabrica[2], 3: fabrica[3]}


<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7f51b55eb450> >

In [13]:
#Executando o modelo
status = modelo_fabrica.Solve()
print("Ótimo: ",status == modelo_fabrica.OPTIMAL)
print("Inviável: ",status == modelo_fabrica.INFEASIBLE)
print("Viável:",status == modelo_fabrica.FEASIBLE)


Ótimo:  True
Inviável:  False
Viável: False


## Avaliando os resultados

In [14]:
print('Custo mínimo para os 1000 carros serem fabricadas: ', int(modelo_fabrica.Objective().Value()))

for i in range(num_fabricas):
    print('fabrica[%i]' % (i), int(fabrica[i].SolutionValue()))


Custo mínimo para os 1000 carros serem fabricadas:  11000
fabrica[0] 250
fabrica[1] 500
fabrica[2] 250
fabrica[3] 0


# Mais um exemplo

A **HotsM** é uma empresa de produtos digitais e com uma gestão financeira muito profissional. Para os próximos 3 anos,m estão previstos um lucro líquido de 200, 250 e 150 milhões de reais.

Eles possuem vários projetos de investimento (participações em outras empresas) e precisam definir quanto e quais projetos tem chance de trazer maior retorno. Par isto, foi calculado o VPL (Valor presente líquido) de cada projeto, veja a tabela com os os dados levantados.

| Projeto de Investimento | Investimentos ano 1 | Investimentos ano 2 | Investimentos ano 3 | VPL do investimento |
|-------------------------|---------------------|---------------------|---------------------|---------------------|
| Projeto 1               | 12                  | 34                  | 12                  | 20                  |
| Projeto 2               | 54                  | 94                  | 67                  | 15                  |
| Projeto 3               | 65                  | 28                  | 49                  | 34                  |
| Projeto 4               | 38                  | 0                   | 8                   | 17                  |
| Projeto 5               | 52                  | 21                  | 42                  | 56                  |
| Projeto 6               | 98                  | 73                  | 25                  | 76                  |
| Projeto 7               | 15                  | 48                  | 53                  | 29                  |

O critério é que caso ela opte por algum projeto eles devem manter a proporcionalidade de aplicações de durante todos os anos (para o ano 1, ano 2 e ano 3 ele deve manter 10% de alocação de investimento em todos os anos.

**A questão é**: qual percentual de cada projeto devo investir para obter o maior VPL?

In [16]:
import numpy as np

In [17]:
#Número de projetos
num_projetos = 7

#Valor total dos projetos por ano
cronograma_desembolso = np.array([[12, 34, 12],
                                  [54, 94, 67],
                                  [65, 28, 49],
                                  [38, 0, 8],
                                  [52, 21, 42],
                                  [98, 73, 25],
                                  [15, 48, 53]])

#VPL dos projetos 
VPL_Projeto = np.array([20, 15, 34, 17, 56, 76, 29])

#Valor máximo de desembolso por ano
Previsto_desembolso = np.array([200 ,250, 150])

## Criando o modelo

In [24]:
# Criando o modelo de investimento
modelo_projetos_inv = pywraplp.Solver('Investimentos HotsM',pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

#Criando as variáveis de decisão
projetos = {}
for i in range(num_projetos):
    projetos[i] = modelo_projetos_inv.NumVar(0, 1, 'projetos[%i]' % (i))
print(projetos)

#Criando a nossa função objetivo
total_VPL =  modelo_projetos_inv.Sum([VPL_Projeto[i] * projetos[i]   for i in range(num_projetos)])    

#Atribuindo a maximização
objetivo = modelo_projetos_inv.Maximize(total_VPL)



#Restrições para os cronogramas de desembolso e projetos

#Restrição do montante para o Ano 1
modelo_projetos_inv.Add(modelo_projetos_inv.Sum(cronograma_desembolso[i,0] *projetos[i]  for i in range(num_projetos)) <= Previsto_desembolso[0])

#Restrição do montante para o Ano 2
modelo_projetos_inv.Add(modelo_projetos_inv.Sum(cronograma_desembolso[i,1] *projetos[i]  for i in range(num_projetos)) <= Previsto_desembolso[1])

#Restrição do montante para o Ano 3
modelo_projetos_inv.Add(modelo_projetos_inv.Sum(cronograma_desembolso[i,2] *projetos[i]  for i in range(num_projetos)) <= Previsto_desembolso[2])



{0: projetos[0], 1: projetos[1], 2: projetos[2], 3: projetos[3], 4: projetos[4], 5: projetos[5], 6: projetos[6]}


<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7f51b2829810> >

## Executando o modelo

In [25]:
#Executando o modelo
modelo_projetos_inv.EnableOutput = True
modelo_projetos_inv.Solve()

print('Tempo: ', modelo_projetos_inv.WallTime(), 'ms')
print('Iterações: ', modelo_projetos_inv.iterations())

Tempo:  2804 ms
Iterações:  8


## Avaliando os resultados

In [26]:
print('Valor máximo do VPL a ser obtido: ', round(modelo_projetos_inv.Objective().Value(),2))

for i in range(num_projetos):
    print('Projeto %i:' % (i+1), round(projetos[i].SolutionValue() *100,2),'%')

Valor máximo do VPL a ser obtido:  193.03
Projeto 1: 100.0 %
Projeto 2: 0.0 %
Projeto 3: 35.38 %
Projeto 4: 0.0 %
Projeto 5: 100.0 %
Projeto 6: 100.0 %
Projeto 7: 100.0 %


# Mais um exemplo

Suponha que você foi contratado como cientista de dados da **757 Mobility Startup** - serviço de assinatura de mobilidade e locomoção.

O primeiro problema para resolver é: dado uma região geográfica (um bairro, por exemplo), você deve calcular em tempo real:
- Número de chamadas de clientes no aplicativo
- Número de motoristas com latitude e longitude dentro da região
- Velocidade média de deslocamento no horário dentro da região
- Atribuir as chamadas para os motoristas de forma que o nível de serviço seja atendido para todos os clientes
- Resolver o problema para reduzir o tempo de espera de todos os envolvidos

## Organizando os dados


In [30]:
#Tempo calculado a partir da ultima atualização de tráfego e posição dos motoristas
Tempo_Previsto = np.array([[90, 76, 75, 70],
                           [35, 85, 55, 65],
                           [125, 95, 90, 105],
                           [45, 110, 95, 115],
                           [60, 105, 80, 75],
                           [45, 65, 110, 95]])

"""Tempo_Previsto = np.array([[90, 76, 75, 70],
                           [35, 85, 55, 65],
                           [125, 95, 90, 105],
                           [45, 110, 95, 115],
                           [60, 105, 80, 75],
                           [45, 65, 110, 95]])
"""

#Número de clientes que solicitaram o serviço no bairro naquele momento
numero_clientes = Tempo_Previsto.shape[0]
numero_motoristas = Tempo_Previsto.shape[1]


## Criando o modelo

In [37]:
# Criando o modelo de atendimento motoristas e clientes
modelo_atendimento = pywraplp.Solver('Modelo de atendimento',pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)


#Criando as variáveis de decisão
atendimento = {}

for i in range(numero_clientes):
    for j in range(numero_motoristas):
        atendimento[i, j] = modelo_atendimento.BoolVar('atendimento[%i,%i]' % (i, j))

#Criando a nossa função objetivo e atribuindo a minimizaçãão
modelo_atendimento.Minimize(modelo_atendimento.Sum(Tempo_Previsto[i,j] * atendimento[i,j] for i in range(numero_clientes) for j in range(numero_motoristas)))


#Restrições para os atendimentos dos clientes e dos motoristas


# Cada cliente deve ser atendido por apenas 1 motorista .
if numero_motoristas < numero_clientes:
    for i in range(numero_clientes):
        modelo_atendimento.Add(modelo_atendimento.Sum([atendimento[i, j] for j in range(numero_motoristas)]) <= 1)
else:
    for i in range(numero_clientes):
        modelo_atendimento.Add(modelo_atendimento.Sum([atendimento[i, j] for j in range(numero_motoristas)]) == 1)
    

# Se o número de clientes for maior que o número de motoristas vamos garantir que os atendimentos mais rápidos aconteçam.
if numero_motoristas < numero_clientes:
    modelo_atendimento.Add(modelo_atendimento.Sum([atendimento[i, j] for i in range(numero_clientes) for j in range(numero_motoristas)]) >= numero_motoristas)

# Cada motorista deve atender no máximo 1 pessoa naquele instante de tempo .
for j in range(numero_motoristas):
    modelo_atendimento.Add(modelo_atendimento.Sum([atendimento[i, j] for i in range(numero_clientes)]) <= 1)


## Executando o modelo

In [38]:
#Executando o modelo
modelo_atendimento.Solve()

print('Tempo: ', modelo_atendimento.WallTime(), 'ms')
print('Iterações: ', modelo_atendimento.iterations())
print('Ótimo: ', modelo_atendimento.OPTIMAL)
print('Factível: ', modelo_atendimento.FEASIBLE)
print('Infactível: ', modelo_atendimento.INFEASIBLE)

Tempo:  1806 ms
Iterações:  0
Ótimo:  0
Factível:  1
Infactível:  2


## Avaliando o modelo

In [39]:
print('Tempo total para os atendimentos: ', round(modelo_atendimento.Objective().Value(),2))

print()
for i in range(numero_clientes):
    for j in range(numero_motoristas):
        #print(atendimento[i, j].solution_value() )
        if atendimento[i, j].solution_value() > 0:
            print('Cliente %d será atendido pelo motorista %d.  Tempo previsto = %d' % (i+1,j+1,Tempo_Previsto[i,j]))
print()


Tempo total para os atendimentos:  235.0

Cliente 1 será atendido pelo motorista 4.  Tempo previsto = 70
Cliente 2 será atendido pelo motorista 3.  Tempo previsto = 55
Cliente 4 será atendido pelo motorista 1.  Tempo previsto = 45
Cliente 6 será atendido pelo motorista 2.  Tempo previsto = 65



# Desafio

Você terminou de implantar o algoritmo de recomendação de produtos para os perfis de clientes na **W2B - E-Commerce**. Você tem certeza da qualidade das recomendações. O algoritmo está com um bom grau de acurácia e precisão. mas você foi chamado para uma reunião com o gerente de estoque do E-Commerce:
- Problema o algoritmo não leva em consideração os níveis de estoque, na hora de fazer a recomendação.
- O bot de atendimento já registrou muitos clientes dizendo que receberam a recomendação mas "nunca tem o produto no estoque"

Sua missão agora é resolver este problema:
- Manter a acurácia das recomendações: o cross selling está funcionando;
- Resolver o problema da insatisfação dos clientes e recomendar o que tem estoque para uma faixa menor de clientes que são potenciais para compra do produto.

Na tabela há uma saída do algoritmo de recomendação para um instante no tempo.

| Clientes  | Produto 1 (score) | Produto 2 (score) | Produto 3 (score) | Produto 4 (score) |
|-----------|-------------------|-------------------|-------------------|-------------------|
| Cliente 1 | 11                | 4                 | 3                 | 9                 |
| Cliente 2 | 3                 | 7                 | 2                 | 3                 |
| Cliente 3 | 4                 | 9                 | 6                 | 5                 |
| Cliente 4 | 5                 | 4                 | 7                 | 7                 |
| Cliente N | ...               | ...               | ...               | ...               |

- Para cada produto você também terá a quantidade em estoque, e este valor significa o número de clientes máximo apra o qual você deve recomendar o produto
- Você deve manter a qualidade do seu algoritmo, mas a sua recomendação agora deve respeitar o estoque, mas de forma que o score geral seja o máximo possível.


# Algoritmo de Recomendação com restrição de Estoque

In [41]:
#Matriz dos escore de clientes 
todos_escores = np.array([ [13,5,10,7,11,1,12,5],
                            [8,7,1,11,11,0,0,1],
                            [4,6,9,8,11,6,7,11],
                            [13,5,11,1,7,10,11,5],
                            [11,9,5,1,6,3,1,3],
                            [9,11,8,14,7,5,6,1],
                            [9,2,5,4,9,0,10,1],
                            [14,2,13,3,10,6,11,10],
                            [6,0,14,5,11,5,1,0],
                            [10,5,0,14,5,9,6,10],       
                            [12,6,11,7,0,13,3,5],
                            [12,7,4,4,13,10,12,14],
                            [9,5,7,1,8,5,9,13],
                            [11,9,5,6,2,1,1,2],
                            [10,1,0,3,13,12,14,7],
                            [3,4,4,0,5,8,1,11],
                            [0,12,2,14,10,14,7,5],
                            [10,12,14,4,4,1,9,11],     
                            [2,6,1,10,11,8,4,4],
                            [11,10,10,6,4,6,5,3]])


#Número de clientes e produtos no instante t
numero_clientes = todos_escores.shape[0]
numero_produtos = todos_escores.shape[1]

print(f"Número de clientes: {numero_clientes}")
print(f"Número de produtos: {numero_produtos}")

#Limites do estoque por produto
Limites_estoque = np.array([2, 1, 3, 0, 10, 4, 2, 4])
#Para simplificar, considere que temos estoque para fazer recomendações para todos os clientes

Número de clientes: 20
Número de produtos: 8


# Criando o modelo

In [42]:
# Criando o modelo de investimento
modelo_recomendacao_estoque = pywraplp.Solver('Modelo para maximizar a recomendação no estoque',pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)


# ...

# Executando o modelo

# Avaliando o modelo

# Anotações finais - Faça você mesmo

Quando devo aplicar análise prescritiva:

- Tarefas muito repetitivas
- Decisões PRECISAM ser tomadas num tempo muito curto
- Atividades passíveis de muitos erros
- Atividades laboriosas e tediosas
- Possuem um largo número de opções de escolha
- Impossível achar  a melhor opção manualmente
- Envolver várias áreas da empresa